In [1]:
# used for manipulating directory paths
import os

# Scientific and vector computation for python
import numpy as np

# Plotting library
from matplotlib import pyplot

# Optimization module in scipy
from scipy import optimize

# will be used to load MATLAB mat datafile format
from scipy.io import loadmat

# library written for this exercise providing additional functions for assignment submission, and others
import utils

# define the submission/grader object for this exercise
grader = utils.Grader()

# tells matplotlib to embed plots within the notebook
%matplotlib inline

In [2]:
#  training data stored in arrays X, y
data = loadmat(os.path.join('Data', 'ex4data1.mat'))
X, y = data['X'], data['y'].ravel()

# set the zero digit to 0, rather than its mapped 10 in this dataset
# This is an artifact due to the fact that this dataset was used in 
# MATLAB where there is no index 0
y[y == 10] = 0

# Number of training examples
m = y.size


In [3]:

# Setup the parameters you will use for this exercise
input_layer_size  = 400  # 20x20 Input Images of Digits
hidden_layer_size = 25   # 25 hidden units
num_labels = 10          # 10 labels, from 0 to 9

# Load the weights into variables Theta1 and Theta2
weights = loadmat(os.path.join('Data', 'ex4weights.mat'))

# Theta1 has size 25 x 401
# Theta2 has size 10 x 26
Theta1, Theta2 = weights['Theta1'], weights['Theta2']

# swap first and last columns of Theta2, due to legacy from MATLAB indexing, 
# since the weight file ex3weights.mat was saved based on MATLAB indexing
Theta2 = np.roll(Theta2, 1, axis=0)

# Unroll parameters 
nn_params = np.concatenate([Theta1.ravel(), Theta2.ravel()])


# DATA SUMMARY 

In [45]:
print('Number of samples =', m)
print('Size of X =', X.shape)
print('Hidden layer size =', hidden_layer_size)
print('The size of Theta1 =', Theta1.shape)
print('The size of Theta2 =', Theta2.shape)

temp_y = np.zeros((m, 10))
for i, num in enumerate(y):
    temp_y[i][num] = 1
print('Size of encoded y =', temp_y.shape)
    
rand_indices = np.random.choice(m, 100, replace=False)
for i in range (10):
    print (temp_y[rand_indices[i]], '~~ y =', y[rand_indices[i]])

Number of samples = 5000
Size of X = (5000, 400)
Hidden layer size = 25
The size of Theta1 = (25, 401)
The size of Theta2 = (10, 26)
Size of encoded y = (5000, 10)
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.] ~~ y = 4
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.] ~~ y = 1
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] ~~ y = 5
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] ~~ y = 3
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0.] ~~ y = 7
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0.] ~~ y = 8
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] ~~ y = 5
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] ~~ y = 3
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] ~~ y = 2
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0.] ~~ y = 3


In [46]:
def sigmoidGradient(z):
    g = np.zeros(z.shape)
    g = utils.sigmoid(z) * (1 - utils.sigmoid(z))
    return g

In [68]:
def checking_funct(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    # Setup some useful variables
    m = y.size
         
    # You need to return the following variables correctly 
    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    
    X = np.concatenate([np.ones((m,1)), X], axis = 1)
    z_2 = X.dot(Theta1.T)
    a_2 = utils.sigmoid(z_2)
    a_2 = np.concatenate([np.ones((m ,1)), a_2], axis = 1)
    h_X = (utils.sigmoid(a_2.dot(Theta2.T)))
    
    delta_3 = h_X - temp_y
    delta_2 = np.delete(delta_3.dot(Theta2), 0, 1)  * sigmoidGradient(z_2)
    
    Theta2_grad = np.delete(delta_3.T.dot(a_2), 0, 1)/m
    Theta1_grad = np.delete(delta_2.T.dot(X), 0, 1)/m
    
    
    print('Shape of delta 2 is', delta_2.shape)
    print('Shape of Theta2_grad is', Theta2_grad.shape)
    print('Shape of Theta1_grad is', Theta1_grad.shape)
    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])
    return grad
    
checking_funct(nn_params,
               input_layer_size,
               hidden_layer_size,
               num_labels,
               X, y, lambda_=0.0)

Shape of delta 2 is (5000, 25)
Shape of Theta2_grad is (10, 25)
Shape of Theta1_grad is (25, 400)


array([0.00000000e+00, 0.00000000e+00, 4.15336892e-09, ...,
       5.00631408e-04, 1.13453370e-03, 1.35707295e-03])

In [89]:
def nnCostFunction(nn_params,
                   input_layer_size,
                   hidden_layer_size,
                   num_labels,
                   X, y, lambda_=0.0):
    Theta1 = np.reshape(nn_params[:hidden_layer_size * (input_layer_size + 1)],
                        (hidden_layer_size, (input_layer_size + 1)))

    Theta2 = np.reshape(nn_params[(hidden_layer_size * (input_layer_size + 1)):],
                        (num_labels, (hidden_layer_size + 1)))

    # Setup some useful variables
    m = y.size
         
    # You need to return the following variables correctly 
    J = 0
    Theta1_grad = np.zeros(Theta1.shape)
    Theta2_grad = np.zeros(Theta2.shape)
    
    temp_y = np.zeros((m, num_labels))
    for i, num in enumerate(y):
        temp_y[i][num] = 1
    
    
    X = np.concatenate([np.ones((m,1)), X], axis = 1)
    z_2 = X.dot(Theta1.T)
    a_2 = utils.sigmoid(z_2)
    a_2 = np.concatenate([np.ones((m ,1)), a_2], axis = 1)
    h_X = (utils.sigmoid(a_2.dot(Theta2.T)))
#     for i in range (m):
#         for k in range (10):
#             J = J + (- temp_y[i][k] * np.log(h_X[i][k]) - (1 - temp_y[i][k]) * np.log(1 - h_X[i][k]))
#     J = J/m

    for i in range (m):
        J = J + (-temp_y[i].dot(np.log(h_X[i].T)) - (1 - temp_y[i]).dot(np.log(1- h_X[i].T)))
    J  = J/m + lambda_ * (np.sum(Theta1[:,1:]**2) + np.sum(Theta2[:, 1:]**2)) / (2 * m)
    
    delta_3 = h_X - temp_y
    delta_2 = np.delete(delta_3.dot(Theta2), 0, 1)  * sigmoidGradient(z_2)
    
    Theta2_grad = (delta_3.T.dot(a_2))/m
    Theta1_grad = (delta_2.T.dot(X))/m
    
    grad = np.concatenate([Theta1_grad.ravel(), Theta2_grad.ravel()])

    return J, grad

In [90]:
lambda_ = 0
J,_ = nnCostFunction(nn_params, input_layer_size, hidden_layer_size,
                   num_labels, X, y, lambda_ = 1)

print('Cost at parameters (loaded from ex4weights): %.6f ' % J)
print('The cost should be about                   : 0.287629.')

Cost at parameters (loaded from ex4weights): 0.383770 
The cost should be about                   : 0.287629.


In [91]:
utils.checkNNGradients(nnCostFunction)

[[-9.27825235e-03 -9.27825236e-03]
 [-3.04978709e-06 -3.04978914e-06]
 [-1.75060082e-04 -1.75060082e-04]
 [-9.62660618e-05 -9.62660620e-05]
 [ 8.89911959e-03  8.89911960e-03]
 [ 1.42869427e-05  1.42869443e-05]
 [ 2.33146358e-04  2.33146357e-04]
 [ 1.17982666e-04  1.17982666e-04]
 [-8.36010761e-03 -8.36010762e-03]
 [-2.59383071e-05 -2.59383100e-05]
 [-2.87468729e-04 -2.87468729e-04]
 [-1.37149709e-04 -1.37149706e-04]
 [ 7.62813551e-03  7.62813551e-03]
 [ 3.69883213e-05  3.69883234e-05]
 [ 3.35320347e-04  3.35320347e-04]
 [ 1.53247077e-04  1.53247082e-04]
 [-6.74798370e-03 -6.74798370e-03]
 [-4.68759764e-05 -4.68759769e-05]
 [-3.76215588e-04 -3.76215587e-04]
 [-1.66560294e-04 -1.66560294e-04]
 [ 3.14544970e-01  3.14544970e-01]
 [ 1.64090819e-01  1.64090819e-01]
 [ 1.64567932e-01  1.64567932e-01]
 [ 1.58339334e-01  1.58339334e-01]
 [ 1.51127527e-01  1.51127527e-01]
 [ 1.49568335e-01  1.49568335e-01]
 [ 1.11056588e-01  1.11056588e-01]
 [ 5.75736493e-02  5.75736493e-02]
 [ 5.77867378e-02  5

<a id="section4"></a>
### 2.4 Backpropagation

![](Figures/ex4-backpropagation.png)

Now, you will implement the backpropagation algorithm. Recall that the intuition behind the backpropagation algorithm is as follows. Given a training example $(x^{(t)}, y^{(t)})$, we will first run a “forward pass” to compute all the activations throughout the network, including the output value of the hypothesis $h_\theta(x)$. Then, for each node $j$ in layer $l$, we would like to compute an “error term” $\delta_j^{(l)}$ that measures how much that node was “responsible” for any errors in our output.

For an output node, we can directly measure the difference between the network’s activation and the true target value, and use that to define $\delta_j^{(3)}$ (since layer 3 is the output layer). For the hidden units, you will compute $\delta_j^{(l)}$ based on a weighted average of the error terms of the nodes in layer $(l+1)$. In detail, here is the backpropagation algorithm (also depicted in the figure above). You should implement steps 1 to 4 in a loop that processes one example at a time. Concretely, you should implement a for-loop `for t in range(m)` and place steps 1-4 below inside the for-loop, with the $t^{th}$ iteration performing the calculation on the $t^{th}$ training example $(x^{(t)}, y^{(t)})$. Step 5 will divide the accumulated gradients by $m$ to obtain the gradients for the neural network cost function.

1. Set the input layer’s values $(a^{(1)})$ to the $t^{th }$training example $x^{(t)}$. Perform a feedforward pass, computing the activations $(z^{(2)}, a^{(2)}, z^{(3)}, a^{(3)})$ for layers 2 and 3. Note that you need to add a `+1` term to ensure that the vectors of activations for layers $a^{(1)}$ and $a^{(2)}$ also include the bias unit. In `numpy`, if a 1 is a column matrix, adding one corresponds to `a_1 = np.concatenate([np.ones((m, 1)), a_1], axis=1)`.

1. For each output unit $k$ in layer 3 (the output layer), set 
$$\delta_k^{(3)} = \left(a_k^{(3)} - y_k \right)$$
where $y_k \in \{0, 1\}$ indicates whether the current training example belongs to class $k$ $(y_k = 1)$, or if it belongs to a different class $(y_k = 0)$. You may find logical arrays helpful for this task (explained in the previous programming exercise).

1. For the hidden layer $l = 2$, set 
$$ \delta^{(2)} = \left( \Theta^{(2)} \right)^T \delta^{(3)} * g'\left(z^{(2)} \right)$$
Note that the symbol $*$ performs element wise multiplication in `numpy`.

1. Accumulate the gradient from this example using the following formula. Note that you should skip or remove $\delta_0^{(2)}$. In `numpy`, removing $\delta_0^{(2)}$ corresponds to `delta_2 = delta_2[1:]`.
$$ \Delta^{(l)} = \Delta^{(l)} + \delta^{(l+1)} (a^{(l)})^{(T)} $$

1. Obtain the (unregularized) gradient for the neural network cost function by dividing the accumulated gradients by $\frac{1}{m}$:
$$ \frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)}$$

<div class="alert alert-box alert-warning">
**Python/Numpy tip**: You should implement the backpropagation algorithm only after you have successfully completed the feedforward and cost functions. While implementing the backpropagation alogrithm, it is often useful to use the `shape` function to print out the shapes of the variables you are working with if you run into dimension mismatch errors.
</div>

[Click here to go back and update the function `nnCostFunction` with the backpropagation algorithm](#nnCostFunction).


**Note:** If the iterative solution provided above is proving to be difficult to implement, try implementing the vectorized approach which is easier to implement in the opinion of the moderators of this course. You can find the tutorial for the vectorized approach [here](https://www.coursera.org/learn/machine-learning/discussions/all/threads/a8Kce_WxEeS16yIACyoj1Q).

In [101]:
a = np.array([[1,3,4], [3,5,6], [2,9,6]])
b = np.array([[1,2],[2,3],[3,4]])
b_ = np.delete(b,0,1)
c = a
c = np.delete(c, 0, 1)
for i in range (3):
    for j in range (3):
        a[i,j] = a[i,j] + 1
print(a)


[[ 2  4  5]
 [ 4  6  7]
 [ 3 10  7]]


In [95]:
<a id="section5"></a>
### 2.5 Regularized Neural Network

After you have successfully implemented the backpropagation algorithm, you will add regularization to the gradient. To account for regularization, it turns out that you can add this as an additional term *after* computing the gradients using backpropagation.

Specifically, after you have computed $\Delta_{ij}^{(l)}$ using backpropagation, you should add regularization using

$$ \begin{align} 
& \frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)} & \qquad \text{for } j = 0 \\
& \frac{\partial}{\partial \Theta_{ij}^{(l)}} J(\Theta) = D_{ij}^{(l)} = \frac{1}{m} \Delta_{ij}^{(l)} + \frac{\lambda}{m} \Theta_{ij}^{(l)} & \qquad \text{for } j \ge 1
\end{align}
$$

Note that you should *not* be regularizing the first column of $\Theta^{(l)}$ which is used for the bias term. Furthermore, in the parameters $\Theta_{ij}^{(l)}$, $i$ is indexed starting from 1, and $j$ is indexed starting from 0. Thus, 

$$
\Theta^{(l)} = \begin{bmatrix}
\Theta_{1,0}^{(i)} & \Theta_{1,1}^{(l)} & \cdots \\
\Theta_{2,0}^{(i)} & \Theta_{2,1}^{(l)} & \cdots \\
\vdots &  ~ & \ddots
\end{bmatrix}
$$

[Now modify your code that computes grad in `nnCostFunction` to account for regularization.](#nnCostFunction)

After you are done, the following cell runs gradient checking on your implementation. If your code is correct, you should expect to see a relative difference that is less than 1e-9.

SyntaxError: invalid syntax (<ipython-input-95-4f80e5e8a5ff>, line 1)